## Tensorflow eager execution implementation

In [102]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print('gpu', gpu)
    tf.config.experimental.set_memory_growth(gpu, True)
    print('memory growth:' , tf.config.experimental.get_memory_growth(gpu))

gpu PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
memory growth: True


In [103]:
import pandas as pd
import numpy as np

train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [104]:
mnist_labels = train_data["label"]
mnist_images = train_data.drop(columns=["label"])

In [105]:
x_train = mnist_images.to_numpy().reshape((42000, 28, 28))
y_train = mnist_labels.to_numpy()
x_test = test_data.to_numpy()

x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

In [106]:
x_train.shape

(42000, 28, 28, 1)

In [107]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(128)
test_ds = tf.data.Dataset.from_tensor_slices((x_test)).batch(128)

In [108]:
train_ds

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>

In [109]:
#model
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(16, 3, activation='relu')
        self.maxp1 = tf.keras.layers.MaxPooling2D()
        self.conv2 = tf.keras.layers.Conv2D(32, 3, activation='relu')
        self.maxp2 = tf.keras.layers.MaxPooling2D()
        self.flatten = tf.keras.layers.Flatten()
        self.d1 = tf.keras.layers.Dense(128, activation='relu')
        self.d2 = tf.keras.layers.Dense(10, activation='softmax')
        
    def call(self, x):
        x = self.conv1(x)
        x = self.maxp1(x)
        x = self.conv2(x)
        x = self.maxp2(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)
    
model = MyModel()

In [110]:
#loss and optimizer

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.RMSprop()

In [111]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [112]:
#define train step and all things registered on gradient tape
def train_step(images, labels):
    with tf.GradientTape() as tape:
        #define our "y"
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients=tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

In [113]:
def test_step(images, labels):
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)
    
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [114]:
#training
EPOCHS = 15

for epoch in range(EPOCHS):
    train_loss.reset_states()
    train_accuracy.reset_states()
#     test_loss.reset_states()
#     test_accuracy.reset_states()
    
    for images, labels in train_ds:
        train_step(images, labels)
        
#     for test_images, test_labels in test_ds:
#         test_step(test_images, test_labels)    
        
    print(
        f'Epoch {epoch+1},'
        f'Loss {train_loss.result()}, '
        f'Accuracy {train_accuracy.result() * 100}, '
#         f'Test Loss: {test_loss.result()},'
#         f'Test Accuracy: {test_accuracy.result() *100}'
    )
        

Epoch 1,Loss 0.2844349145889282, Accuracy 91.4952392578125, 
Epoch 2,Loss 0.07451262325048447, Accuracy 97.6595230102539, 
Epoch 3,Loss 0.049257028847932816, Accuracy 98.44762420654297, 
Epoch 4,Loss 0.03670252859592438, Accuracy 98.88571166992188, 
Epoch 5,Loss 0.028798161074519157, Accuracy 99.1738052368164, 
Epoch 6,Loss 0.022534480318427086, Accuracy 99.36190032958984, 
Epoch 7,Loss 0.01776924915611744, Accuracy 99.5047607421875, 
Epoch 8,Loss 0.014079555869102478, Accuracy 99.63333892822266, 
Epoch 9,Loss 0.011605693958699703, Accuracy 99.71190643310547, 
Epoch 10,Loss 0.00904617179185152, Accuracy 99.77143096923828, 
Epoch 11,Loss 0.008362174034118652, Accuracy 99.78095245361328, 
Epoch 12,Loss 0.006639446597546339, Accuracy 99.83809661865234, 
Epoch 13,Loss 0.005950248334556818, Accuracy 99.81904602050781, 
Epoch 14,Loss 0.005145372822880745, Accuracy 99.87619018554688, 
Epoch 15,Loss 0.004134430084377527, Accuracy 99.88095092773438, 


In [115]:
x_test.shape

(28000, 784, 1)

In [116]:
preds = model.predict(x_test.reshape((28000, 28, 28, 1)))

In [117]:
solution = [np.argmax(v) for i, v in enumerate(preds)]

In [118]:
npsol = np.array(solution)
print(len(npsol))

28000


In [119]:
index = np.array([i+1 for i in range(len(npsol))])

In [120]:
index

array([    1,     2,     3, ..., 27998, 27999, 28000])

In [121]:
result = np.column_stack((index, npsol))

In [122]:
df = pd.DataFrame(result,columns=['ImageId', 'Label'])


In [123]:
df.to_csv("submission.csv", index=False)